In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Step 1: Install Required Libraries

In [ ]:
import os
import sys

In [ ]:
try:
    import ultralytics
    print("ultralytics is already installed.")
except ImportError:
    print("Installing ultralytics...")
    os.system(f"{sys.executable} -m pip install ultralytics")

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

## Step 2: Explore Dataset

In [ ]:
folder_path = '/content/drive/MyDrive/comp vision/clips'

if os.path.isdir(folder_path):
    print(f"Contents of {folder_path}:")
    for item in os.listdir(folder_path)[:10]:  # Show first 10 items
        print(item)
    print("... (truncated)")
else:
    print(f"Error: Folder '{folder_path}' not found.")

file_counts = {
    'play': 0,
    'background': 0,
    'challenge': 0,
    'other': 0
}

if os.path.isdir(folder_path):
    for item in os.listdir(folder_path):
        if item.startswith('play-'):
            file_counts['play'] += 1
        elif item.startswith('background-'):
            file_counts['background'] += 1
        elif item.startswith('challenge-'):
            file_counts['challenge'] += 1
        else:
            file_counts['other'] += 1

    print("File counts:")
    for key, value in file_counts.items():
        print(f"{key}: {value}")
else:
    print(f"Error: Folder '{folder_path}' not found.")

In [ ]:
## Step 3: Extract Frames from Videos

# Define paths
clips_path = Path(folder_path)
output_path = Path('extracted_frames')
output_path.mkdir(exist_ok=True)

In [ ]:
# Create subfolders for categories
categories = ['play', 'background', 'challenge']
for cat in categories:
    (output_path / cat).mkdir(exist_ok=True)

In [ ]:
# Function to extract frames
def extract_frames(video_path, output_dir, frame_interval=30):
    cap = cv2.VideoCapture(str(video_path))
    count = 0
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frame_filename = f"{video_path.stem}_frame_{frame_count}.jpg"
            cv2.imwrite(str(output_dir / frame_filename), frame)
            frame_count += 1
        count += 1
    cap.release()
    print(f"Extracted {frame_count} frames from {video_path.name}")

# Extract frames from all videos
for video_file in clips_path.glob('*.mp4'):
    if video_file.name.startswith('play-'):
        output_dir = output_path / 'play'
    elif video_file.name.startswith('background-'):
        output_dir = output_path / 'background'
    elif video_file.name.startswith('challenge-'):
        output_dir = output_path / 'challenge'
    else:
        continue  # Skip other categories for now
    extract_frames(video_file, output_dir)

print("Frame extraction completed.")

In [ ]:
## Step 4: Prepare Dataset for Annotation

# Note: Manual annotation required here. Use LabelImg or Roboflow to annotate the extracted frames.
# After annotation, place labels in corresponding folders as .txt files in YOLO format.

# For now, we'll assume annotations are done and proceed to dataset splitting.

# List all images
all_images = []
for cat in categories:
    cat_path = output_path / cat
    images = list(cat_path.glob('*.jpg'))
    all_images.extend(images)

print(f"Total extracted images: {len(all_images)}")

In [ ]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO('yolov8m.pt')

# Train the model
results = model.train(
    data='/content/drive/MyDrive/Computer Vision/My-First-Project-2/data.yaml',     # Path to your YAML file
    epochs=50,            # Increase if you have more data/time
    imgsz=640,            # Image size
    batch=8,              # Adjust based on your GPU
    name='football_yolov8n'  # Folder name for saving results
)

In [ ]:
from IPython.display import Image
Image(filename='runs/detect/football_yolov8n/results.png', width=600)

In [ ]:
model.val()

In [ ]:
results = model.predict(source='/content/drive/MyDrive/Computer Vision/throwin-3418_1.mp4', show=True, save=True)

In [ ]:
import shutil
import os

source_path = '/content/My-First-Project-2'
destination_path = '/content/drive/MyDrive/Computer Vision'

try:
    shutil.move(source_path, destination_path)
    print(f"'{source_path}' moved to '{destination_path}' successfully.")
except FileNotFoundError:
    print(f"Error: '{source_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")